In [1]:
import pandas as pd
import sentencepiece as spm
import numpy as np
import os
from konlpy.tag import Mecab
import glob

In [2]:
dir_path = os.getenv("HOME") +'/aiffel/DATA'

In [3]:
df_train=pd.read_csv(dir_path + '/train_data_0324.csv', index_col='Unnamed: 0')
df_test=pd.read_csv(dir_path + '/test_data_0324.csv', index_col='Unnamed: 0')

In [4]:
len(df_test)

125

In [5]:
df_train.head()

,reg,topic,stdn,dial,eng
0,jd,역사,생각이 쪼금씩 바뀌더라고,생각이 쪼금씩 바뀌드라고,I've changed my mind a little bit.
1,jd,가족,어 알겠는가 외국인들이 그래도 잘 적응하고,어 알겄는가 애국인들이 그또 잘 적응하고,You know what? Foreigners still get used to it.
2,jd,건강,아으 갑자기 예전에 맹장 뜯은 게 생각난다.,아으 갑자기 에전에 맹장 뜯은 게 생각난디야.,"All of a sudden, I think I've ripped off my ap..."
3,jd,스타일,보면은 조금 품위 있게 나이 들어가야 되겠단 생각을 참 많이 해요.,보먼은 쫌 품위 있게 나이 들어가야 되겠단 싱각을 참 많이 해요잉.,I have a lot of ideas about getting older in a...
4,jd,먹거리,약간 조금 약간 맛이 쪼끔 거시기 하긴 한데 그래도,약간 쫌 약간 맛이 쪼끔 머시기 하긴 한디 그또,"It's a little bit of a tastey, but still."


In [19]:
vc = df_train.eng.values +' '+df_train.dial.values

In [20]:
mecab = Mecab()

In [21]:
import tqdm
cm_dial = []
for i in tqdm.tqdm(df_train.dial.values) :
    temp = ' '.join(mecab.morphs(i))
    cm_dial.append(temp)

100%|████████████████████████████████| 948219/948219 [00:43<00:00, 22031.35it/s]


In [35]:
def generate_tokenizer(corpus, vocab_size, lang="en", pad_id=0, bos_id=1, eos_id=2, unk_id=3):

    temp_file = os.getenv('HOME') + f'/aiffel/DATA/corpus_{lang}.txt'     # corpus를 받아 txt파일로 저장
    
    with open(temp_file, 'w') as f:
        for row in corpus:
            f.write(str(row) + '\n')
    
    # Sentencepiece를 이용해 
    spm.SentencePieceTrainer.Train(
        f'--input={temp_file} --pad_id={pad_id} --bos_id={bos_id} --eos_id={eos_id} \
        --unk_id={unk_id} --model_prefix=spm_{lang} --vocab_size={vocab_size} --model_type=unigram \
        --user_defined_symbols=<jj>,<jd>,<gs>,<cc>,<kw>,<std>'   # model_r1
    )
    tokenizer = spm.SentencePieceProcessor()
    tokenizer.Load(f'spm_{lang}.model') # model_r1

    return tokenizer

In [23]:
# one_tokenizer = generate_tokenizer(vc, 16009, lang ='enc_spm16000')

In [24]:
enc_tokenizer = generate_tokenizer(df_train.eng.values, 16009, lang ='enc_spm16000')

In [37]:
dec_tokenizer = generate_tokenizer(df_train.dial.values, 16009, lang ='dec_spm16000')

In [34]:
enc_tokenizer.encode_as_pieces('뭐라카노')

['▁', '뭐라카노']

In [6]:
# dir_path = os.getenv("HOME") + '/aiffel/saturi/notebook/Model/'

enc_tokenizer = spm.SentencePieceProcessor()
enc_tokenizer.Load('spm_enc_spm16000.model')

dec_tokenizer = spm.SentencePieceProcessor()
dec_tokenizer.Load('spm_dec_msp16000.model')

dec_tokenizer1 = spm.SentencePieceProcessor()
dec_tokenizer1.Load('spm_dec_cmsp16000.model')

dec_tokenizer.set_encode_extra_options("bos:eos")

True

In [7]:
df_train['eng'] = '<'+df_train['reg']+'> ' + df_train['eng']
df_test['eng'] = '<'+df_test['reg']+'> ' + df_test['eng']

In [8]:
df_train.head()

,reg,topic,stdn,dial,eng
0,jd,역사,생각이 쪼금씩 바뀌더라고,생각이 쪼금씩 바뀌드라고,<jd> I've changed my mind a little bit.
1,jd,가족,어 알겠는가 외국인들이 그래도 잘 적응하고,어 알겄는가 애국인들이 그또 잘 적응하고,<jd> You know what? Foreigners still get used ...
2,jd,건강,아으 갑자기 예전에 맹장 뜯은 게 생각난다.,아으 갑자기 에전에 맹장 뜯은 게 생각난디야.,"<jd> All of a sudden, I think I've ripped off ..."
3,jd,스타일,보면은 조금 품위 있게 나이 들어가야 되겠단 생각을 참 많이 해요.,보먼은 쫌 품위 있게 나이 들어가야 되겠단 싱각을 참 많이 해요잉.,<jd> I have a lot of ideas about getting older...
4,jd,먹거리,약간 조금 약간 맛이 쪼끔 거시기 하긴 한데 그래도,약간 쫌 약간 맛이 쪼끔 머시기 하긴 한디 그또,"<jd> It's a little bit of a tastey, but still."


 0.   toks_en     
 1.   toks_dec    
 2.   source_txt  
 3.   target_txt  
 4.   topic       
 5.   reg         

In [9]:
txt = df_train.dial.values[0]
txt

'생각이 쪼금씩 바뀌드라고 '

In [10]:
# spm
dec_tokenizer.encode_as_pieces(txt)

['<s>', '▁생각', '이', '▁쪼금', '씩', '▁바뀌', '드', '라고', '</s>']

In [11]:
# spm
dec_tokenizer1.encode_as_pieces(txt)

['▁생각', '이', '▁쪼금', '씩', '▁바뀌', '드라고']

In [12]:
# # msp
# dec_tokenizer_msp.encode_as_pieces(df_train.dial.values[0])

In [13]:
# # custom msp
# dec_tokenizer.encode_as_pieces(df_train.dial.values[0])

In [14]:
import tqdm
def tokenize_data(df) :
    dial_text = df['dial'].values
    eng_text = df['eng'].values

    toks_en = []
    toks_dec = []

    for i in tqdm.tqdm(range(len(df))) :
        
        en_tokenized = enc_tokenizer.encode(eng_text[i])
        dial_tokenized = dec_tokenizer.encode(dial_text[i])
        toks_en.append(en_tokenized)
        toks_dec.append(dial_tokenized)
    
    df['toks_en'] = toks_en
    df['toks_dec'] = toks_dec

In [15]:
for i in [df_test, df_train]:
    tokenize_data(i)

100%|████████████████████████████████| 948219/948219 [00:51<00:00, 18423.67it/s]


In [34]:
df_train.head(2)

,reg,topic,stdn,dial,eng,toks_en,toks_dec
0,jd,역사,생각이 쪼금씩 바뀌더라고,생각이 쪼금씩 바뀌드라고,<jd> <jd> I've changed my mind a little bit.,"[15951, 5, 15951, 5, 13, 15974, 36, 923, 105, ...","[1, 98, 14511, 692, 14855, 2053, 1175, 2]"
1,jd,가족,어 알겠는가 외국인들이 그래도 잘 적응하고,어 알겄는가 애국인들이 그또 잘 적응하고,<jd> <jd> You know what? Foreigners still get ...,"[15951, 5, 15951, 5, 117, 143, 166, 15978, 132...","[1, 17, 109, 14807, 14515, 14512, 9265, 14550,..."


In [16]:
dir_path = os.getenv("HOME") +'/aiffel/DATA/pickle'
os.makedirs(dir_path, exist_ok=True)
df_train.to_pickle(dir_path + '/data_train_msp_16000_0324.pkl','gzip')
df_test.to_pickle(dir_path + '/data_test_msp_16000_0324.pkl','gzip')